Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
tf.__version__

'1.3.0'

In [3]:
pickle_file = '/home/victor/data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [31]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    #tf.reset_default_graph()
    
    # Input data.
    with tf.name_scope('data'):
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    with tf.name_scope("Weights"):
        layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev=0.1))
        layer1_biases = tf.Variable(tf.zeros([depth]))
        layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev=0.1))
        layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
        layer3_weights = tf.Variable(tf.truncated_normal(
          [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
        layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
        layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_labels], stddev=0.1))
        layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        with tf.name_scope("Conv1"):
            conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer1_biases)
            tf.summary.histogram("Weights_1", layer1_weights)
            tf.summary.histogram("Biases_1", layer1_biases)
            tf.summary.histogram("Activations_1", conv)
            tf.summary.histogram("relu_1", hidden)
        with tf.name_scope("Conv2"):
            conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer2_biases)
            tf.summary.histogram("Weights_2", layer2_weights)
            tf.summary.histogram("Biases_2", layer2_biases)
            tf.summary.histogram("Activations_2", conv)
            tf.summary.histogram("relu_2", hidden)
        with tf.name_scope("Reshape_1"):
            shape = hidden.get_shape().as_list()
            reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
            tf.summary.histogram("Weights_1", reshape)
        with tf.name_scope("Hidden_layer"):
            hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
            tf.summary.histogram("Weights_3", layer3_weights)
            tf.summary.histogram("Biases_3", layer3_biases)
            tf.summary.histogram("Activations_hidden", hidden)
            tf.summary.histogram("relu_hidden", hidden)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    with tf.name_scope("Loss"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
        tf.summary.scalar("CrossEntropy", loss)

    # Optimizer.
    with tf.name_scope("Train"):
        optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    summaries = tf.summary.merge_all()
    # Predictions for the training, validation, and test data.
    #with tf.name_scope('accuracy'):
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [34]:
num_steps = 1001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    writer = tf.summary.FileWriter("/home/victor/tensorboard")
    writer.add_graph(session.graph)
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        summ, _, l, predictions = session.run(
          [summaries, optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            writer.add_summary(summ, global_step=step)
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                   valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.336174
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.773899
Minibatch accuracy: 37.5%
Validation accuracy: 52.1%
Minibatch loss at step 100: 0.663217
Minibatch accuracy: 87.5%
Validation accuracy: 70.2%
Minibatch loss at step 150: 0.391982
Minibatch accuracy: 87.5%
Validation accuracy: 75.9%
Minibatch loss at step 200: 1.026265
Minibatch accuracy: 68.8%
Validation accuracy: 78.0%
Minibatch loss at step 250: 0.570650
Minibatch accuracy: 81.2%
Validation accuracy: 78.1%
Minibatch loss at step 300: 0.642091
Minibatch accuracy: 68.8%
Validation accuracy: 79.4%
Minibatch loss at step 350: 0.831813
Minibatch accuracy: 75.0%
Validation accuracy: 78.2%
Minibatch loss at step 400: 0.476570
Minibatch accuracy: 81.2%
Validation accuracy: 79.6%
Minibatch loss at step 450: 0.303255
Minibatch accuracy: 93.8%
Validation accuracy: 79.8%
Minibatch loss at step 500: 0.799423
Minibatch accuracy: 75.0%
Validation accuracy: 80.5%
M

In [ ]:
!tensorboard --logdir=/home/victor/tensorboard

TensorBoard 0.1.5 at http://lagex03:6006 (Press CTRL+C to quit) 

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [18]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
   
  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # FIRST convolution. Changed the stride in convolution from 2 to 1
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    # Max pooling of stride 2 and kernel size 2.
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1] , [1, 2, 2, 1], padding='SAME')
    # SECOND convolution. Changed the stride in convolution from 2 to 1
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    # Max pooling of stride 2 and kernel size 2.
    pool = tf.nn.max_pool(hidden,[1, 2, 2, 1] , [1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    pool = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(pool, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.760408
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.814209
Minibatch accuracy: 50.0%
Validation accuracy: 37.1%
Minibatch loss at step 100: 1.276543
Minibatch accuracy: 43.8%
Validation accuracy: 66.3%
Minibatch loss at step 150: 1.353961
Minibatch accuracy: 62.5%
Validation accuracy: 75.2%
Minibatch loss at step 200: 0.575695
Minibatch accuracy: 75.0%
Validation accuracy: 77.2%
Minibatch loss at step 250: 0.327457
Minibatch accuracy: 100.0%
Validation accuracy: 79.2%
Minibatch loss at step 300: 0.519972
Minibatch accuracy: 87.5%
Validation accuracy: 80.2%
Minibatch loss at step 350: 1.032627
Minibatch accuracy: 68.8%
Validation accuracy: 79.9%
Minibatch loss at step 400: 0.452348
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
Minibatch loss at step 450: 0.252455
Minibatch accuracy: 93.8%
Validation accuracy: 82.5%
Minibatch loss at step 500: 0.359451
Minibatch accuracy: 87.5%
Validation accuracy: 82.1%
M

In [19]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.852317
Minibatch accuracy: 6.2%
Validation accuracy: 14.6%
Minibatch loss at step 50: 1.780191
Minibatch accuracy: 37.5%
Validation accuracy: 45.2%
Minibatch loss at step 100: 0.980480
Minibatch accuracy: 56.2%
Validation accuracy: 67.4%
Minibatch loss at step 150: 1.463324
Minibatch accuracy: 62.5%
Validation accuracy: 73.7%
Minibatch loss at step 200: 0.522175
Minibatch accuracy: 75.0%
Validation accuracy: 76.8%
Minibatch loss at step 250: 0.310000
Minibatch accuracy: 93.8%
Validation accuracy: 80.0%
Minibatch loss at step 300: 0.667721
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Minibatch loss at step 350: 0.958886
Minibatch accuracy: 68.8%
Validation accuracy: 79.1%
Minibatch loss at step 400: 0.428243
Minibatch accuracy: 93.8%
Validation accuracy: 81.6%
Minibatch loss at step 450: 0.398518
Minibatch accuracy: 81.2%
Validation accuracy: 82.0%
Minibatch loss at step 500: 0.326389
Minibatch accuracy: 87.5%
Validation accuracy: 82.6%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

-----
## Resumo dos resultados problema 1 e 2
Em todos os casos mostrados na tabela foi usado *max pooling*(`tf.nn.max_pool`)


|      Rede         |   Iterações   | Minibatch accuracy(%) (last step)  | Validation accuracy(%) (last step) | Test accuracy(%) |
|:-----------------:|:-------------:|:-------------------------------:|:-------------------------------:|:-------------:|
| 2 conv + 1 hidden |      1001     |            75.0                 |             85.0                |      91.4     |
| 2 conv + 1 hidden |      2001     |            100.0                |             86.5                |      92.3     |
| 2 conv + dropout + 1 hidden |      1001     |            68.8                 |             82.4                |      89.3     |
| 2 conv + dropout + 1 hidden  |      2001     |            93.8                 |             84.8                |      90.5     |
| 2 conv + dropout + 2 hidden  |      1001     |            81.2                 |             81.8                |      89.1     |
| 2 conv + dropout + 2 hidden  |      2001     |            100.0                 |             84.9                |      91.2     |


## Versão com Dropout adicionado

In [10]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, keep_prob):
    # FIRST convolution. Changed the stride in convolution from 2 to 1
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    # Max pooling of stride 2 and kernel size 2.
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1] , [1, 2, 2, 1], padding='SAME')
    # SECOND convolution. Changed the stride in convolution from 2 to 1
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    # Max pooling of stride 2 and kernel size 2.
    pool = tf.nn.max_pool(hidden,[1, 2, 2, 1] , [1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    pool = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    #return tf.matmul(pool, layer4_weights) + layer4_biases
    drop = tf.nn.dropout(pool, keep_prob)
    return tf.matmul(drop, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset, 0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

In [11]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.233912
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.460403
Minibatch accuracy: 0.0%
Validation accuracy: 19.1%
Minibatch loss at step 100: 2.024372
Minibatch accuracy: 31.2%
Validation accuracy: 27.8%
Minibatch loss at step 150: 1.971875
Minibatch accuracy: 31.2%
Validation accuracy: 49.3%
Minibatch loss at step 200: 1.272959
Minibatch accuracy: 50.0%
Validation accuracy: 65.2%
Minibatch loss at step 250: 1.098244
Minibatch accuracy: 62.5%
Validation accuracy: 63.1%
Minibatch loss at step 300: 0.972910
Minibatch accuracy: 81.2%
Validation accuracy: 76.7%
Minibatch loss at step 350: 1.878827
Minibatch accuracy: 50.0%
Validation accuracy: 74.0%
Minibatch loss at step 400: 0.986826
Minibatch accuracy: 62.5%
Validation accuracy: 79.5%
Minibatch loss at step 450: 0.458380
Minibatch accuracy: 87.5%
Validation accuracy: 78.8%
Minibatch loss at step 500: 0.462214
Minibatch accuracy: 93.8%
Validation accuracy: 78.6%
Min

## O dobro de iterações.

In [14]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.923190
Minibatch accuracy: 6.2%
Validation accuracy: 14.0%
Minibatch loss at step 50: 2.130001
Minibatch accuracy: 18.8%
Validation accuracy: 18.5%
Minibatch loss at step 100: 2.160818
Minibatch accuracy: 18.8%
Validation accuracy: 44.6%
Minibatch loss at step 150: 1.361556
Minibatch accuracy: 56.2%
Validation accuracy: 60.6%
Minibatch loss at step 200: 1.271626
Minibatch accuracy: 50.0%
Validation accuracy: 66.6%
Minibatch loss at step 250: 0.855745
Minibatch accuracy: 75.0%
Validation accuracy: 74.8%
Minibatch loss at step 300: 1.088580
Minibatch accuracy: 56.2%
Validation accuracy: 72.7%
Minibatch loss at step 350: 1.514741
Minibatch accuracy: 56.2%
Validation accuracy: 69.9%
Minibatch loss at step 400: 0.606890
Minibatch accuracy: 81.2%
Validation accuracy: 77.3%
Minibatch loss at step 450: 0.684161
Minibatch accuracy: 75.0%
Validation accuracy: 79.5%
Minibatch loss at step 500: 0.703629
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Mi

## Adição de uma camada fully conected ao final (parâmetros idênticos ao da rede anterior). 

In [15]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, keep_prob):
    # FIRST convolution. Changed the stride in convolution from 2 to 1
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    # Max pooling of stride 2 and kernel size 2.
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1] , [1, 2, 2, 1], padding='SAME')
    # SECOND convolution. Changed the stride in convolution from 2 to 1
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    # Max pooling of stride 2 and kernel size 2.
    pool = tf.nn.max_pool(hidden,[1, 2, 2, 1] , [1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    pool = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    drop = tf.nn.dropout(pool, keep_prob)
    drop = tf.matmul(drop, layer4_weights) + layer4_biases
    return tf.matmul(drop, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset, 0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

In [13]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.447222
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.077196
Minibatch accuracy: 18.8%
Validation accuracy: 27.9%
Minibatch loss at step 100: 1.998652
Minibatch accuracy: 31.2%
Validation accuracy: 42.3%
Minibatch loss at step 150: 2.209246
Minibatch accuracy: 25.0%
Validation accuracy: 57.4%
Minibatch loss at step 200: 1.076287
Minibatch accuracy: 68.8%
Validation accuracy: 65.4%
Minibatch loss at step 250: 0.817762
Minibatch accuracy: 62.5%
Validation accuracy: 72.2%
Minibatch loss at step 300: 1.028095
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 350: 1.380752
Minibatch accuracy: 50.0%
Validation accuracy: 73.8%
Minibatch loss at step 400: 0.598163
Minibatch accuracy: 87.5%
Validation accuracy: 78.5%
Minibatch loss at step 450: 0.698779
Minibatch accuracy: 75.0%
Validation accuracy: 79.8%
Minibatch loss at step 500: 0.657879
Minibatch accuracy: 81.2%
Validation accuracy: 78.1%
Mi

## O dobro de iterações

In [16]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.510982
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.370649
Minibatch accuracy: 0.0%
Validation accuracy: 15.1%
Minibatch loss at step 100: 1.856747
Minibatch accuracy: 31.2%
Validation accuracy: 46.9%
Minibatch loss at step 150: 2.033961
Minibatch accuracy: 25.0%
Validation accuracy: 59.6%
Minibatch loss at step 200: 1.140025
Minibatch accuracy: 56.2%
Validation accuracy: 65.3%
Minibatch loss at step 250: 1.380067
Minibatch accuracy: 56.2%
Validation accuracy: 74.4%
Minibatch loss at step 300: 0.910214
Minibatch accuracy: 62.5%
Validation accuracy: 75.5%
Minibatch loss at step 350: 1.293463
Minibatch accuracy: 56.2%
Validation accuracy: 68.8%
Minibatch loss at step 400: 0.593660
Minibatch accuracy: 81.2%
Validation accuracy: 78.0%
Minibatch loss at step 450: 0.517397
Minibatch accuracy: 87.5%
Validation accuracy: 78.9%
Minibatch loss at step 500: 0.569346
Minibatch accuracy: 75.0%
Validation accuracy: 80.1%
Min